# Avoided crossing with asymmetric quantum well

[Go back to index](./index.ipynb)

[https://en.wikipedia.org/wiki/Avoided_crossing](https://en.wikipedia.org/wiki/Avoided_crossing)

In quantum physics and quantum chemistry, an avoided crossing is the phenomenon where two eigenvalues of an Hermitian matrix representing a quantum observable and depending on N continuous real parameters cannot become equal in value ("cross") except on a manifold of N-2 dimensions.

In [1]:
%matplotlib widget
from pylab import axes, setp, axis, xlabel, ylabel, title, plot, subplot, draw, connect, show
from numpy import linspace, sqrt, ones, arange, diag, argsort, zeros
from scipy.linalg import eig
import matplotlib.pyplot as plt
from math import pi
from numpy import *


def potential(x, mu):
    """Potentialfunktion fuer  Doppelmulde, mu: Parameter"""
    return x**4 - x**2 + mu*x # asymmetrische Doppelmulde


def diagonalisierung(hquer, L, N, pot=potential, mu = 0.0):
    """Berechne sortierte Eigenwerte und zugehoerige Eigenfunktionen.

       Eingabe:
         hquer: effektives hquer
         L: legt betrachtetes Intervall [-L,L] fest
         N: Zahl der Gitterpunkte, d.h. Groesse der Matrix
         pot: Potentialfunktion der Form pot(x, mu)
         mu: Potentialparameter
       Ausgabe:
         ew: sortierte Eigenwerte (Array der Laenge N)
         ef: entsprechend sortierte Eigenvektoren, ef[:,i] (Groesse N*N)
         x: Ortsgitter (Array der Laenge N)
         dx: Ortsgitterabstand
         V: Potential an den Stellen x (Array der Laenge N)
    """
    x = linspace(-L, L, N+2)[1:N+1]               # Gitterpunkte 
    dx = x[1] - x[0]                              # Gitterabstand
    V = pot(x, mu)
    z = hquer**2 /2.0/dx**2                       # Nebendiagonalen
    h = (diag(V+2.0*z) + diag(-z*ones(N-1), -1)   # Aufstellen der
                      + diag(-z*ones(N-1), 1) )   #  Hamilton-Matrix

    ew, ef = eig(h)                               # Diagonalisierung
    ew = ew.real                                  # Realteil der Eigenwerte
    ind = argsort(ew)                             # Indizes f. sort. Array
    ew = ew[ind]                                  # Sortieren der ew nach ind
    ef = ef[:, ind]                               # Sortieren der Spalten von
                                                  #  ef nach ind
    ef = ef/sqrt(dx)                              # richtige Normierung 
    return ew, ef, x, dx, V


def plot_eigenfunktionen(ax, ew, ef, x, V, width=1, Emax=0.1, fak=0.009):
    """Plot der niedrigsten Eigenfunktionen 'ef' im Potential 'V(x)'
       auf Hoehe der Eigenwerte 'ew' in den Plotbereich 'ax'.
       
       Der optionale Parameter 'width' (mit Defaultwert 1)
       gibt die Linienstaerke beim Plot der Eigenfunktionen
       an. 'width' kann auch ein Array von Linienstaerken sein.
       'Emax' (mit Default-Wert V_0/10) legt die Energieobergrenze
       fuer den Plot fest.
       'fak' ist ein Skalierungsfaktor fuer die graphische Darstellung
       der Eigenfunktionen.
    """
    ax[1].plot(x, V, c='k', linewidth=1.0)  
    ax[1].set_xlim([min(x), max(x)])
    ax[1].set_ylim([min(V), Emax])
    
    ax[1].yaxis.set_label_position('right')
    ax[1].yaxis.tick_right()
    ax[1].set_xticks([-1.5, -1.0, -0.5, 0.0, 0.5, 1.0, 1.5])
    ax[1].set_xticklabels([-1.5, -1.0, -0.5, 0.0, 0.5, 1.0, 1.5])
    ax[1].set_xlabel(r'$x/a$', fontsize = 10)
    ax[1].set_ylabel(r'$V(x)/V_0\ \rm{, Eigenfunctions\ with\ Eigenvalues}$', fontsize = 10)
      
    indmax = sum(ew<=Emax)                       
    colors = ['b', 'g', 'r', 'c', 'm', 'y']      
    if not hasattr(width, "__iter__"):           
        width = width*ones(indmax)               
    for i in arange(indmax):                     
        ax[1].plot(x, fak*abs(ef[:, i])**2+ew[i], linewidth=width[i]+.1, color=colors[i%len(colors)])


def plot_zeitentwicklung(ax, ew, ef, x, V, coeff, zeiten): 
    """ Berechne die Zeitentwicklung eines Wellenpakets,
        bestimmt durch die Entwicklungskoeffizienten `coeff`.
        Uebergeben werden der Plotbereich `ax`, die Eigenwerte `ew`,
        die Eigenfunktionen `ef`, das Potential `V` an den Orten `x`
        und das Zeitarray `zeiten`.
    """
    
    E0_qm = dot(abs(coeff)**2, ew)                   # qm. Energieerwartung
    fak = 0.01                                       # Plot-Skalierungsfaktor
    phi_t0 = dot(ef, coeff*exp(-1j*ew*zeiten[0]/hquer))  # Zeitentwicklung
                                                     # Plot mit Linienbreite
                                                     #  gleich Anregungsstaerke
    phi = dot(ef, coeff*exp(-1j*ew*zeiten[-1]/hquer))
    ax[1].plot(x, E0_qm+fak*abs(phi)**2, 'r--', linewidth=2.0)
#     draw()                                           # Plot fuer t=0    
#     for zeit in zeiten[1:len(zeiten)]:               # Plot fuer t > 0
#         phi = dot(ef, coeff*exp(-1j*ew*zeit/hquer))  # Zeitentwicklung
#         setp(wellenpaket[0], ydata=E0_qm+fak*abs(phi)**2) # Plot Wellenpaket
#         draw()


In [2]:
from ipywidgets import FloatSlider, jslink, VBox, HBox

mu = 0.06                                            # Potentialparameter
L = 1.5                                              # x-Bereich ist [-L,L]
N = 200                                              # Zahl der Gitterpunkte
hquer = 0.06                                         # effektives hquer
sigma_x = 0.1                                        # Breite Gauss
zeiten = linspace(0.0, 10.0, 400)                    # Zeiten f. Zeitentw.


smu = FloatSlider(value = 0.0, min = -0.1, max = 0.1, step = 0.01, description = r'$\mu$: ')
                                                         
fig, ax = plt.subplots(1, 2, figsize=(20, 8))

fig.suptitle('Numerial Solution of One Dimension Schroedinger Equation', fontsize = 10)

mu1 = []
ew1 = []
ew2 = []
for i in linspace(-0.1, 0.1, 50):
    ew, ef, x, dx, V = diagonalisierung(hquer, L, N, mu = i)
    mu1.append(i)
    ew1.append(ew[0])
    ew2.append(ew[1])

ew, ef, x, dx, V = diagonalisierung(hquer, L, N, mu = 0.0)

ax[0].plot(mu1, ew1, c='b')
ax[0].plot(mu1, ew2, c='g')

s1 = ax[0].plot(0.0, ew[0], 'ro')
s2 = ax[0].plot(0.0, ew[1], 'yo')

ax[0].set_xlim([min(mu1), max(mu1)])
ax[0].set_ylim([min(ew1), max(ew2)])

ax[0].set_xlabel(r'$\mu$ value', fontsize = 10)
ax[0].set_ylabel(r'Two lowest eigenvalues$', fontsize = 10)
ax[0].set_xticks([-0.1, -0.05, 0.0, 0.05, 0.1])
ax[0].set_xticklabels([-0.1, -0.05, 0.0, 0.05, 0.1])

plot_eigenfunktionen(ax, ew, ef, x, V)


def on_mu_change(change):
    for i in range(len(ax[1].lines)):
        ax[1].lines.pop(0)
    ew, ef, x, dx, V = diagonalisierung(hquer, L, N, mu = smu.value)
    plot_eigenfunktionen(ax, ew, ef, x, V)
    
    ax[0].lines.pop(-1)
    ax[0].lines.pop(-1)
    ax[0].plot(smu.value, ew[1], 'yo', label = str(format(ew[1], '.2f')))
    ax[0].plot(smu.value, ew[0], 'ro', label = str(format(ew[0], '.2f')))
    ax[0].legend()

smu.observe(on_mu_change, names = 'value')

def on_press(event):
    for i in range(len(ax[1].lines)):
        ax[1].lines.pop(0)
    ew, ef, x, dx, V = diagonalisierung(hquer, L, N, mu = smu.value)
    plot_eigenfunktionen(ax, ew, ef, x, V)
    phi0 = (2.0*pi*sigma_x**2)**(-0.25)*exp(-(x-event.xdata)**2.0/(4.0*sigma_x**2))
    coeff = dot(conjugate(transpose(ef)), phi0)*dx
    plot_zeitentwicklung(ax, ew, ef, x, V, coeff, zeiten)

cid = fig.canvas.mpl_connect('button_press_event', on_press)


display(HBox([smu]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

This work has been done with the support of the EPFL Open Science Fund [OSSCAR](http://www.osscar.org).

<img src="http://www.osscar.org/wp-content/uploads/2019/03/OSSCAR-logo.png" style="height:40px; width: 200px"/>